# Assignment BDM

# ISB ID: 71710035

In [167]:
sqlContext

In [85]:
rdd_users = sc.textFile('file:///home/cloudera/Desktop/Assignment/users.csv')

In [86]:
rdd_transactions = sc.textFile('file:///home/cloudera/Desktop/Assignment/transactions.csv')

In [87]:
user = rdd_users.map(lambda x: x.split(",")).toDF(['Userid', 'Emailid',\
                                                   'NativeLanguage','Location'])


In [95]:
transaction = rdd_transactions.map(lambda x: x.split(",")).toDF(['TransactionID',\
                                                                 'ProductID'\
                                                                ,'Userid', 'Price', \
                                                                 'Productdecription'])

### a)	Count of unique locations where each product is sold. 

In [89]:
# Performing Inner Join on two dataframe of users and transaction
loc = user.join(transaction, ["Userid"], 'inner')
loc.collect()

[Row(Userid=u'1', Emailid=u'user1@company.com', NativeLanguage=u'ES', Location=u'MX', TransactionID=u'11', ProductID=u'1004', Price=u'129', Productdecription=u'whatchamacallit'),
 Row(Userid=u'2', Emailid=u'user4@domain.com', NativeLanguage=u'EN', Location=u'US', TransactionID=u'12', ProductID=u'1004', Price=u'129', Productdecription=u'whatchamacallit'),
 Row(Userid=u'3', Emailid=u'user5@company.com', NativeLanguage=u'FR', Location=u'FR', TransactionID=u'5', ProductID=u'1003', Price=u'89', Productdecription=u'gadget'),
 Row(Userid=u'5', Emailid=u'user12@service.io', NativeLanguage=u'EN', Location=u'CA', TransactionID=u'13', ProductID=u'1005', Price=u'199', Productdecription=u'doohickey'),
 Row(Userid=u'6', Emailid=u'user17@website.net', NativeLanguage=u'FR', Location=u'FR', TransactionID=u'10', ProductID=u'1003', Price=u'89', Productdecription=u'gadget'),
 Row(Userid=u'7', Emailid=u'user21@company.com', NativeLanguage=u'FR', Location=u'FR', TransactionID=u'14', ProductID=u'1004', Price

In [140]:
# Count by Location
count_loc = loc.groupBy('Location').count()
count_loc.show()

+--------+-----+
|Location|count|
+--------+-----+
|      MX|    4|
|      US|    5|
|      IN|    1|
|      CA|    2|
|      FR|    3|
+--------+-----+



In [136]:
# Since there is some confusion regarding grouping by product or location.  
#This is an attempt to group by product
product = loc.map(lambda y: ((int(y[5])), (y[3]))).toDF(['ProductID','Location']).distinct()

In [144]:
grouping= product.groupBy("ProductID")
grouping.count().show()

+---------+-----+
|ProductID|count|
+---------+-----+
|     1001|    2|
|     1002|    2|
|     1003|    1|
|     1004|    3|
|     1005|    1|
+---------+-----+



### b)	Find out products bought by each user.

In [134]:
# b) Find out products bought by each user.
loc.select('Userid', 'Productdecription').show()

+------+-----------------+
|Userid|Productdecription|
+------+-----------------+
|     1|  whatchamacallit|
|     2|  whatchamacallit|
|     3|           gadget|
|     5|        doohickey|
|     6|           gadget|
|     7|  whatchamacallit|
|     9|      thingamajig|
|    10|      thingamajig|
|    16|            gizmo|
|    17|  whatchamacallit|
|    19|  whatchamacallit|
|    19|            gizmo|
|    22|      thingamajig|
|    26|            gizmo|
|    30|            gizmo|
+------+-----------------+



### c)	Total spending done by each user on each product.

In [158]:
price_change= loc.map(lambda y: ((int(y[0])), float(y[6])))

In [159]:
print (price_change.collect())

[(1, 129.0), (2, 129.0), (3, 89.0), (5, 199.0), (6, 89.0), (7, 129.0), (9, 99.0), (10, 99.0), (16, 149.0), (17, 129.0), (19, 129.0), (19, 149.0), (22, 99.0), (26, 149.0), (30, 149.0)]


In [297]:
#c)	Total spending done by each user on each product.

sum_price = price_change.reduceByKey(lambda x,y: x+y)
type(sum_price)


pyspark.rdd.PipelinedRDD

In [164]:
sum_price.toDF(['userid', 'TotalPrice']).show()

+------+----------+
|userid|TotalPrice|
+------+----------+
|     1|     129.0|
|     2|     129.0|
|     3|      89.0|
|     5|     199.0|
|     6|      89.0|
|     7|     129.0|
|     9|      99.0|
|    10|      99.0|
|    16|     149.0|
|    17|     129.0|
|    19|     278.0|
|    22|      99.0|
|    26|     149.0|
|    30|     149.0|
+------+----------+



# Question 2 

In [21]:
df = sqlContext.jsonFile("file:///home/cloudera/Desktop/Assignment/tweets.json")
df.show()

+--------------------+------------------+-----------------+--------------------+-------------------+
|             country|                id|            place|                text|               user|
+--------------------+------------------+-----------------+--------------------+-------------------+
|               India|572692378957430785|           Orissa|@always_nidhi @Yo...|    Srkian_nishu :)|
|       United States|572575240615796737|        Manhattan|@OnlyDancers Bell...| TagineDiningGlobal|
|       United States|572575243883036672|        Claremont|1/ "Without the a...|        Daniel Beer|
|       United States|572575252020109313|           Vienna|idk why people ha...|   someone actually|
|       United States|572575274539356160|           Boston|Taste of Iceland!...|     BostonAttitude|
|       United States|572647819401670656|          Suwanee|Know what you don...|Collin A. Zimmerman|
|           Indonesia|572647831053312000|      Mario Riawa|Serasi ade haha @...|   Rinie Sy

### Printing Schema for the Data Frame

In [22]:
df.printSchema()

root
 |-- country: string (nullable = true)
 |-- id: string (nullable = true)
 |-- place: string (nullable = true)
 |-- text: string (nullable = true)
 |-- user: string (nullable = true)



In [23]:
# caching the data form
df.registerTempTable("df")
df.cache()

DataFrame[country: string, id: string, place: string, text: string, user: string]

### Get all of the tweets made by a user (any user would work. We should be able to replace user names to get tweets by that particular user)

In [24]:
input_user = raw_input("Please enter the user name:")
print input_user
sqlContext.sql("SELECT text, user as handler FROM df WHERE user LIKE '{}'".format(input_user)).show()

Please enter the user name:stefan
stefan
+--------------------+-------+
|                text|handler|
+--------------------+-------+
|@traceyb65 I'm up...| stefan|
+--------------------+-------+



### Find count of all tweets by each user.

In [19]:
#c)	Find count of all tweets by each user.

sqlContext.sql("SELECT user, count(1) as NumberofTweets FROM df GROUP BY \
user ORDER BY NumberofTweets DESC").show(5)

+--------------------+--------------+
|                user|NumberofTweets|
+--------------------+--------------+
|       #QuissyUpSoon|           258|
|Inès Mendes Askiip ♥|           185|
|           #4Rentinc|           100|
|                  MV|            58|
|    williampriceking|            46|
+--------------------+--------------+
only showing top 5 rows



### Get a list of all of the people who are mentioned in tweets.

In [71]:
#d)	Get a list of all of the people who are mentioned in tweets.
sqlContext.sql("SELECT regexp_extract(text,'@([A-Za-z0-9_]+)') as person FROM df \
WHERE text LIKE '%@%'").show()

+--------------+
|        person|
+--------------+
|  always_nidhi|
|   OnlyDancers|
|IcelandNatural|
|       AdeRais|
|      BeezyDH_|
|  blakeshelton|
|  DjGregStreet|
|     traceyb65|
|     hokkazonn|
|       daunugh|
|        3Will1|
| 4fucksakesmag|
|  XAmbassadors|
|    Ashton5SOS|
|  jamiisonkent|
|    Parisblass|
|     __akgrown|
|              |
|     Jazzfeezy|
|   NBCTheVoice|
+--------------+
only showing top 20 rows



### Count the number of time each person is mentioned in the entire dataset of tweets.

In [73]:
sqlContext.sql("SELECT regexp_extract(text,'@([A-Za-z0-9_]+)') as person_intweets, count(*)\
 as count FROM df WHERE text like '%@%' GROUP BY text").show()

+---------------+-----+
|person_intweets|count|
+---------------+-----+
|servaescolton16|    1|
|     NICKIMINAJ|    1|
| mitasukamdiyah|    1|
|   ThatsSarcasm|    1|
|ComedyWorIdStar|    1|
|      lecamping|    1|
|         MAEJOR|    1|
|  Lange_Records|    1|
|        DJSWHiT|    1|
|       sadables|    1|
| itstonybennett|    1|
|    ParisHilton|    1|
|     em__singer|    1|
|     JayRansiky|    1|
|        brikeey|    1|
|    _CharquellC|    1|
|          jhene|    1|
|    ShawnMendes|    1|
|      VAME_2015|    1|
|   NiykeeHeaton|    1|
+---------------+-----+
only showing top 20 rows



### Give top 50 users who are mentioned the most.

In [28]:
#f)	Give top 50 users who are mentioned the most.

sqlContext.sql("SELECT user, count(*) as Top50 FROM df GROUP BY user ORDER BY Top50 DESC").show(50)

+--------------------+-----+
|                user|Top50|
+--------------------+-----+
|       #QuissyUpSoon|  258|
|Inès Mendes Askiip ♥|  185|
|           #4Rentinc|  100|
|                  MV|   58|
|    williampriceking|   46|
|✌ Follow Me MAEJOR ✌|   44|
|    Phillthy McNasty|   43|
|       K.O.H.O.R.T.S|   41|
|  #AMNT KINGTAECRAZY|   41|
|        Ghafla.co.ke|   36|
|        Ully U Music|   35|
|            Codeclic|   33|
|           Lord Dash|   30|
|  TagineDiningGlobal|   30|
|      Herri Setiawan|   29|
|          Dell Feddi|   29|
|   Kidrauhl Forever❤|   25|
|     Trendsmap Paris|   23|
|      #TurnYaSneakUp|   22|
|                Bel |   19|
|   Change Barcelona!|   18|
|          alec reyes|   17|
|          Lil Shadow|   17|
|           arianator|   15|
|Music Dir. F.Herrera|   14|
|       davidfelician|   13|
|  Terra Rising Films|   12|
|ST★RS RADIO 91,6 MHz|   12|
|         TheBeachKid|   12|
|   Ecotour.com Deals|   11|
|        Lionel Rigal|   11|
|     Naijaloa

In [76]:
#Top 50 person mentioned in tweets
sqlContext.sql("SELECT regexp_extract(text,'@([A-Za-z0-9_]+)') as person, count(*) as count FROM df  WHERE \
text like '%@%' GROUP BY text ORDER BY count(*) DESC").distinct().show(50)

+---------------+-----+
|         person|count|
+---------------+-----+
|     NICKIMINAJ|    2|
|       MeekMill|    2|
|            GMA|    2|
|     1goodtexan|    2|
|servaescolton16|    1|
|     NICKIMINAJ|    1|
| mitasukamdiyah|    1|
|   ThatsSarcasm|    1|
|ComedyWorIdStar|    1|
|      lecamping|    1|
|         MAEJOR|    1|
|  Lange_Records|    1|
|        DJSWHiT|    1|
|       sadables|    1|
| itstonybennett|    1|
|    ParisHilton|    1|
|     em__singer|    1|
|     JayRansiky|    1|
|        brikeey|    1|
|    _CharquellC|    1|
|          jhene|    1|
|    ShawnMendes|    1|
|      VAME_2015|    1|
|   NiykeeHeaton|    1|
| TheeDeanAugust|    1|
|     AshleyJudd|    1|
|               |    1|
|      MszDeevah|    1|
|          kfury|    1|
|  ChrisMastamp3|    1|
|   ComptonMovie|    1|
|      infoyogya|    1|
|          train|    1|
|     AsifAAhmad|    1|
|       WatchGHS|    1|
|  DisneylandDTD|    1|
|    jesusss_123|    1|
|NYCFoodieFinder|    1|
|        Jim_B60

### Get a list of all hashtags mentioned in the dataset.

In [77]:
#g)	Get a list of all hashtags mentioned in the dataset.

hashtag = sqlContext.sql("SELECT regexp_extract(text,'#([A-Za-z0-9_]+)') as \
person FROM df WHERE text LIKE '%#%'")
hashtag.show(5)

+--------------+
|        person|
+--------------+
|shutUpAndDANCE|
|        SHINee|
|          JR50|
|           DME|
|    nowplaying|
+--------------+
only showing top 5 rows



### Find how many times each hashtag is mentioned in the dataset.

In [83]:
#h)	Find how many times each hashtag is mentioned in the dataset.

sqlContext.sql("SELECT regexp_extract(text,'#([A-Za-z0-9_]+)') as handlers, count(*)\
 as count FROM df WHERE text like '%#%' GROUP BY text").distinct().show()

+--------------------+-----+
|            handlers|count|
+--------------------+-----+
|        RumiPlaylist|    1|
|                  NO|    1|
|             Detroit|    1|
|                 art|    1|
|               REACH|    1|
|    papoDeAtleticano|    1|
|        bdscertified|    1|
|      DetroitStandUp|    1|
|TourAlbumBergerak...|    1|
|           education|    1|
|              Studio|    1|
|            fainting|    1|
|          FlyAmerica|    1|
|            lemonsup|    1|
|          NowPlaying|    1|
|             blessed|    1|
|                Tour|    1|
|                NeYo|    1|
|         SabineWeiss|    1|
|            podscast|    1|
+--------------------+-----+
only showing top 20 rows



### Get a list of all of the people who are located in a particular city (e.g. Paris)

In [25]:
#i)	Get a list of all of the people who are located in a particular city (e.g. Paris)
city = raw_input("Please enter the city's name to know how many user belongs to the city:")
print city
sqlContext.sql("SELECT text, user, place  FROM df WHERE place='{}'".format(city)).show()

Please enter the city's name to know how many user belongs to the city:Paris
Paris
+--------------------+--------------------+-----+
|                text|                user|place|
+--------------------+--------------------+-----+
|'pivert', 'usager...|     Trendsmap Paris|Paris|
|@jhornain Découvr...|            Codeclic|Paris|
|Bon mardi ! OCTAV...|        EPHYRE Paris|Paris|
|@YFEFRANCE Hey! Y...|  Helene PASQUALETTI|Paris|
|日曜日の朝のパリ市庁舎前のスケート...|     Takaaki Kishida|Paris|
|#Passing the #caf...|    Michael S. Osman|Paris|
|I'm at ESP - Sall...|     Marie-Charlotte|Paris|
|Grand Oral IT Nig...|    Juliette Laridan|Paris|
|Etiopathe à Clama...| Boscheron Etiopathe|Paris|
|エッフェル塔綺麗♪♪ @ Torr...|                 たいき|Paris|
|Photos de @cinemi...|   Maxence d'Aubigny|Paris|
|I'm at Avenue de ...|        Lionel Rigal|Paris|
|Business meetings...|         Foued KEFIF|Paris|
|#WakeUp #BelleJou...|                 Wil|Paris|
|Paris: ☼ Temp: 3°...|         FranceBidet|Paris|
|Fing, @la_fing e

###  Get country wise distribution of users, and find out which country ranks highest in terms of number of tweets, and number of users.

In [36]:
sqlContext.sql("SELECT country, count(DISTINCT user) as user, count(text) as \
tweet FROM df GROUP BY country ORDER BY count(user) DESC").show()

+--------------------+----+-----+
|             country|user|tweet|
+--------------------+----+-----+
|       United States|3595| 4841|
|              France| 377|  737|
|           Indonesia| 278|  370|
|      United Kingdom| 301|  365|
|              Brasil| 183|  256|
|              Canada| 154|  172|
|Republika ng Pili...| 127|  151|
|           Argentina|  57|  104|
|        South Africa|  73|   92|
|           Australia|  77|   90|
|               India|  56|   66|
|              México|  47|   59|
|                  日本|  52|   57|
|              España|  32|   53|
|            Malaysia|  37|   50|
|               Kenya|   9|   44|
|             Türkiye|  36|   42|
|               Ghana|   9|   38|
|            Colombia|  27|   33|
|             Nigeria|  22|   33|
+--------------------+----+-----+
only showing top 20 rows



### 2K Find out number of tweets where a user is from France and mentions Paris in their tweets.

In [65]:
sqlContext.sql("SELECT *  FROM df WHERE country='France' and text LIKE '%aris%'").show()


+-------+------------------+--------------------+--------------------+--------------------+
|country|                id|               place|                text|                user|
+-------+------------------+--------------------+--------------------+--------------------+
| France|572626703064952833|Allondrelle-la-Ma...|@Ashton5SOS be mo...|CrazyRockCountryLady|
| France|572704775713628161|               Paris|'pivert', 'usager...|     Trendsmap Paris|
| France|572685132475305984|               Paris|@jhornain Découvr...|            Codeclic|
| France|572678334687150080|               Paris|Bon mardi ! OCTAV...|        EPHYRE Paris|
| France|572664931973242880|              Chessy|Happy campers! #D...|       Candice LeRae|
| France|572672815926796288|               Paris|@YFEFRANCE Hey! Y...|  Helene PASQUALETTI|
| France|572648068832755712|         Saint-Avold|Opéra Garnier Par...|      Dominique Lang|
| France|572557043942461440|              Chessy|#Disneyland #Pari...|      saif

# Question 3

In [3]:
#Build Graph
from pyspark.sql.functions import *
from graphframes import *


### 3A Creating a GraphFrame

In [4]:
trip = sqlContext.read.format("com.databricks.spark.csv").options(header='true', inferschema='true').load("file:///home/cloudera/Desktop/Assignment/trip.csv")

In [5]:
trip.registerTempTable("trip")
trip.cache()
trip1 = trip.withColumnRenamed("Start Station", "src").distinct()
tripEdges = trip1.withColumnRenamed("End Station", "dst").distinct()
tripEdges = tripEdges.select("src","dst")
tripEdges.show(5)

+--------------------+--------------------+
|                 src|                 dst|
+--------------------+--------------------+
|      Market at 10th|     2nd at Townsend|
|Palo Alto Caltrai...|       Park at Olive|
|    San Pedro Square|SJSU - San Salvad...|
|   Market at Sansome|     Beale at Market|
|       Market at 4th|      Market at 10th|
+--------------------+--------------------+
only showing top 5 rows



In [6]:
station = sqlContext.read.format("com.databricks.spark.csv").options(header='true', inferschema='true').load("file:///home/cloudera/Desktop/Assignment/station.csv")

In [7]:
station.registerTempTable("station")
station.cache()
stationVertices = station.withColumnRenamed("name", "id").distinct()
stationVertices = stationVertices.select("id")
stationVertices.show(5)

+--------------------+
|                  id|
+--------------------+
| San Salvador at 1st|
|SJSU 4th at San C...|
|Grant Avenue at C...|
|California Ave Ca...|
|Harry Bridges Pla...|
+--------------------+
only showing top 5 rows



In [8]:
g = GraphFrame(stationVertices, tripEdges)
print g

GraphFrame(v:[id: string], e:[src: string, dst: string])


### 3BFind out number of incoming connections and outgoing connections for each node and print the top 10 nodes

In [17]:
# b)	Find out number of incoming connections and outgoing connections for each node 
#and print the top 10 nodes.
ind = g.inDegrees.sort(desc("inDegree"))
ind.show(10)

+--------------------+--------+
|                  id|inDegree|
+--------------------+--------+
|San Francisco Cal...|   34810|
|San Francisco Cal...|   22523|
|Harry Bridges Pla...|   17810|
|     2nd at Townsend|   15463|
|     Townsend at 7th|   15422|
|Embarcadero at Sa...|   15065|
|   Market at Sansome|   13916|
|   Steuart at Market|   13617|
|Temporary Transba...|   12966|
|  Powell Street BART|   10239|
+--------------------+--------+
only showing top 10 rows



In [18]:
# b)	Find out number of incoming connections and outgoing connections for each node 
#and print the top 10 nodes.
out = g.outDegrees.sort(desc("outDegree"))
out.show(10)

+--------------------+---------+
|                  id|outDegree|
+--------------------+---------+
|San Francisco Cal...|    26304|
|San Francisco Cal...|    21758|
|Harry Bridges Pla...|    17255|
|Temporary Transba...|    14436|
|Embarcadero at Sa...|    14158|
|     2nd at Townsend|    14026|
|     Townsend at 7th|    13752|
|   Steuart at Market|    13687|
|      Market at 10th|    11885|
|   Market at Sansome|    11431|
+--------------------+---------+
only showing top 10 rows



### 3C Find out which are the most common direct routes that people take and print top 10.

In [41]:
#c)	Find out which are the most common direct routes that people take and print top 10.

motifs = g.find("(a)-[directroute] -> (b)")

In [42]:
motifs.groupby("a","b").count().sort((desc("count"))).show(10)

+--------------------+--------------------+-----+
|                   a|                   b|count|
+--------------------+--------------------+-----+
|[San Francisco Ca...|   [Townsend at 7th]| 3748|
|[Harry Bridges Pl...|[Embarcadero at S...| 3145|
|   [2nd at Townsend]|[Harry Bridges Pl...| 2973|
|   [Townsend at 7th]|[San Francisco Ca...| 2734|
|[Harry Bridges Pl...|   [2nd at Townsend]| 2640|
|[Embarcadero at F...|[San Francisco Ca...| 2439|
| [Steuart at Market]|   [2nd at Townsend]| 2356|
|[Embarcadero at S...| [Steuart at Market]| 2330|
|   [Townsend at 7th]|[San Francisco Ca...| 2192|
|[Temporary Transb...|[San Francisco Ca...| 2184|
+--------------------+--------------------+-----+
only showing top 10 rows



### 3D From the analysis in b, see which are the stations where people most frequently start their trips but do not come back. (Hint: You might have to think of incoming connections as a ratio of outgoing connections). Print top 10 such stations.

In [21]:
#d)	From the analysis in b, see which are the stations where people most frequently 
#start their trips but do not come back. (Hint: You might 
#have to think of incoming connections as a ratio of outgoing connections). Print top 10 such stations.


df_join = ind.join(out, ['id'])
df_join = df_join.withColumn('Start-End', df_join['outDegree']/df_join['inDegree'])
df_join.sort(desc('Start-End')).show()

+--------------------+--------+---------+------------------+
|                  id|inDegree|outDegree|         Start-End|
+--------------------+--------+---------+------------------+
|Grant Avenue at C...|    4319|     8337|1.9303079416531606|
|       2nd at Folsom|    4727|     7999|1.6921937804104084|
|Powell at Post (U...|    4134|     6425|  1.55418480890179|
|          Mezes Park|     145|      212|1.4620689655172414|
|Evelyn Park and Ride|     725|      978|1.3489655172413793|
|     Beale at Market|    6330|     8359|1.3205371248025277|
| Golden Gate at Polk|    2852|     3646|1.2784011220196354|
|         Ryland Park|     880|     1120|1.2727272727272727|
|San Francisco Cit...|    1627|     2052|1.2612169637369393|
|Palo Alto Caltrai...|     900|     1116|              1.24|
|  San Jose City Hall|     671|      832| 1.239940387481371|
|South Van Ness at...|    4879|     5876|1.2043451526952245|
|Redwood City Publ...|      98|      118|1.2040816326530612|
|      Market at 10th|  

### 3E Find all such patterns where any station a is connected to station b, b is connected to c, but c is not directly connected to a.

In [11]:
edge = tripEdges.distinct()
v = stationVertices.distinct()

In [13]:
graph_df = GraphFrame(v,edge)
graph_df

GraphFrame(v:[id: string], e:[src: string, dst: string])

In [14]:
#e)	Find all such patterns 
#where any station a is connected to station b, b is connected to c, but c is not directly connected to a.
pattern1= graph_df.find("(a)-[] -> (b); (b)-[]->(c); !(c)-[]->(a)")

In [16]:
## First 10 rows..
pattern1.distinct().show(10)

+--------------------+--------------------+--------------------+
|                   a|                   b|                   c|
+--------------------+--------------------+--------------------+
| [Franklin at Maple]|        [Mezes Park]|        [Mezes Park]|
|[San Jose Diridon...|[San Jose Civic C...|[Mountain View Ca...|
|[Rengstorff Avenu...|[Castro Street an...|     [Howard at 2nd]|
|[San Francisco Ca...|[San Francisco Ca...|[San Antonio Calt...|
|   [Townsend at 7th]|     [Market at 4th]|[Stanford in Redw...|
|[Temporary Transb...|   [Beale at Market]|        [Mezes Park]|
|[Grant Avenue at ...|   [Beale at Market]|        [Mezes Park]|
|[Embarcadero at B...|[Powell Street BART]|[San Antonio Shop...|
|[California Ave C...|[University and E...|[San Francisco Ca...|
|     [Park at Olive]|[Palo Alto Caltra...|[Stanford in Redw...|
+--------------------+--------------------+--------------------+
only showing top 10 rows



### Run a PageRank algorithm to figure out which is the most important station in the entire graph

In [15]:
results = g.pageRank(resetProbability=0.15, tol=0.01)
results.vertices.show()

+--------------------+-------------------+
|                  id|           pagerank|
+--------------------+-------------------+
|San Jose Civic Ce...| 0.7850066002667163|
|Redwood City Calt...| 2.0126980700870134|
|San Mateo County ...| 0.7055535181356853|
|Rengstorff Avenue...|0.44132804966440414|
|South Van Ness at...|  0.552295911128788|
|           Japantown| 0.8100868826750073|
|    San Pedro Square|  1.222063710529912|
|San Francisco Cit...|0.27914858766684636|
|     Spear at Folsom|   0.55331286744544|
|San Francisco Cal...| 1.9712236933192793|
|Evelyn Park and Ride| 0.5361583702600942|
|Paseo de San Antonio| 0.8866333561586521|
|University and Em...| 1.1957276186540653|
|Santa Clara Count...| 0.4570804856379124|
|    Adobe on Almaden|0.47928670715376825|
|  San Jose City Hall| 0.6157118631337893|
|Embarcadero at Sa...| 1.3681133631508233|
|San Antonio Caltr...| 0.9900409779366691|
|   Market at Sansome| 1.2280531103488006|
|       St James Park| 0.6591598371119667|
+----------

In [16]:
# Top Station
results.vertices.sort(desc("pagerank")).show()

+--------------------+------------------+
|                  id|          pagerank|
+--------------------+------------------+
|San Jose Diridon ...|3.5710727174436583|
|San Francisco Cal...|2.9548304501454092|
|Mountain View Cal...|2.2733423294327504|
|Redwood City Calt...|2.0126980700870134|
|San Francisco Cal...|1.9712236933192793|
|Harry Bridges Pla...|1.6188585199762089|
|     2nd at Townsend|1.4055254623119484|
|Santa Clara at Al...| 1.393972040450786|
|     Townsend at 7th|1.3893781153627542|
|Embarcadero at Sa...|1.3681133631508233|
|   Steuart at Market|1.3032272474235531|
|Palo Alto Caltrai...|1.2639463072306332|
|   Market at Sansome|1.2280531103488006|
|    San Pedro Square| 1.222063710529912|
|Temporary Transba...|1.2140210475942863|
|University and Em...|1.1957276186540653|
|Mountain View Cit...|1.0979929184245107|
|San Antonio Shopp...|1.0251906345486324|
|Stanford in Redwo...|1.0224956580157227|
|San Antonio Caltr...|0.9900409779366691|
+--------------------+------------